In [5]:
import os
import torch
from dotenv import load_dotenv
from ultralytics import YOLO
from roboflow import Roboflow

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/home/khairuddin/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'.


In [6]:
load_dotenv()
api_key = os.getenv('API_KEY')

In [7]:
HOME = os.getcwd()

datasets_dir = os.path.join(HOME, "datasets")

if not os.path.exists(datasets_dir):
    os.makedirs(datasets_dir)

os.chdir(datasets_dir)

In [8]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("thworkspace").project("car-detection-isouk")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to car-detection-1 in yolov8:: 100%|██████████| 306/306 [00:00<00:00, 5468.22it/s]


In [9]:
data_path = dataset.location
data_yaml = os.path.join(data_path, 'data.yaml')

In [10]:
model = YOLO('yolov8n.pt')

100.0%


In [11]:
# Hyperparameters
epochs = 20
batch_size = 8
img_size = 640

In [12]:
# Train model
model.train(data=data_yaml,
            epochs=epochs,
            batch=batch_size,
            imgsz=img_size,
            plots=True)

New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.12.3 torch-2.3.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/khairuddin/Kuliah/Computer Vision/FP/parking-spot/datasets/car-detection-1/data.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_s

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x75ec636309e0>
fitness: 0.7795180861196441
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.75629])
names: {0: 'car'}
plot: True
results_dict: {'metrics/precision(B)': 0.9835382673741974, 'metrics/recall(B)': 0.94836443572128, 'metrics/mAP50(B)': 0.9885852181804407, 'metrics/mAP50-95(B)': 0.7562884047795555, 'fitness': 0.7795180861196441}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.960091749827067, 'inference': 64.96779123942058, 'loss': 9.934107462565105e-05, 'postprocess': 17.696539560953777}

In [19]:
best_model = YOLO('runs/detect/train/weights/best.pt')

metrics = best_model.val(
    data=data_yaml,
    split='test'
)

Ultralytics YOLOv8.0.196 🚀 Python-3.12.3 torch-2.3.1+cu121 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /home/khairuddin/Kuliah/Computer Vision/FP/parking-spot/datasets/car-detection-1/test/labels... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<00:00, 2265.35it/s]
val: New cache created: /home/khairuddin/Kuliah/Computer Vision/FP/parking-spot/datasets/car-detection-1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.60it/s]
                   all          6        116      0.933      0.931      0.978      0.749
Speed: 0.9ms preprocess, 68.7ms inference, 0.0ms loss, 20.6ms postprocess per image
Results saved to runs/detect/val
